In [ ]:
# IMPORTS

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random
from tqdm import tqdm
from numba import njit

# DEFINE PARAMETERS AND LOAD DATA

# Parameters
size = 6

# Load grid positions
grid_paths = np.loadtxt('../data/grid_paths.txt')

### Create Base Network

In [ ]:
# Generate all possible grid positions
positions = [(i, j) for i in range(size) for j in range(size)]

# Create the base network
G_base = nx.Graph()
nodes = list(range(size * size))
G_base.add_nodes_from(nodes)

# Assign properties (polarity and location) to each node
for node in nodes:
    G_base.nodes[node]['polarity'] = ''  # Initialize polarity as empty
    G_base.nodes[node]['location'] = positions[node]

# Create a list of edges manually (for corners, extremes, and central nodes)
edges = []

# Corners
edges.append([0, 0 + 1])
edges.append([0, 0 + size])
edges.append([0 + size - 1, 0 + size - 2])
edges.append([0 + size - 1, 0 + size + size - 1])
edges.append([size * size - size, size * size - 2 * size])
edges.append([size * size - size, size * size - size + 1])
edges.append([size * size - 1, size * size - 2])
edges.append([size * size - 1, size * size - size - 1])

# Extremes
# Right lateral
for i in range(2, size):
    edges.append([size * i - 1, size * i - 2])
    edges.append([size * i - 1, size * i - 1 + size])
    edges.append([size * i - 1, size * i - 1 - size])
# Left lateral
for i in range(1, size - 1):
    edges.append([size * i, size * i + 1])
    edges.append([size * i, size * i + size])
    edges.append([size * i, size * i - size])
# Top edge
for i in range(1, size - 1):
    edges.append([i, i + 1])
    edges.append([i, i - 1])
    edges.append([i, i + size])
# Bottom edge
for i in range(1, size - 1):
    edges.append([size * (size - 1) + i, size * (size - 1) + i + 1])
    edges.append([size * (size - 1) + i, size * (size - 1) + i - 1])
    edges.append([size * (size - 1) + i, size * (size - 1) + i - size])
    
# Centrals
for i in range(1, size - 1):
    for j in range(1, size - 1):
        index = size * i + j
        edges.append([index, index + 1])
        edges.append([index, index - 1])
        edges.append([index, index + size])
        edges.append([index, index - size])

# Convert edges to weighted edges (initialize all weights as 0)
weighted_edges = [(i, j, {'weight': 0}) for i, j in edges]
G_base.add_edges_from(weighted_edges)

In [ ]:
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base, 'location')
nx.draw(G_base, pos, with_labels=True, node_color='skyblue', node_size=1500,
        font_size=12, font_weight='bold')
plt.title('Base Network')
plt.show()

In [ ]:
def plot_configuration(conf_number, G_base, grid_paths, node_size=700, figsize=(5, 5), dpi=100):
    """
    Plot a specific path configuration in the network.
    
    Parameters:
        conf_number : int
            Configuration index to plot.
        G_base : networkx.Graph
            Base graph structure.
        grid_paths : array-like
            Array of all possible paths.
        node_size : int, optional
            Size of nodes in the plot.
        figsize : tuple, optional
            Size of the figure.
        dpi : int, optional
            Figure resolution.
            
    Returns:
        G : networkx.Graph
            A copy of the base graph with the configuration plotted.
    """
    # Create a copy of the base graph
    G = G_base.copy()
    
    # Retrieve the path for the given configuration
    path = grid_paths[conf_number]
    
    # Update edge weights along the path (note: converting from 1-based to 0-based indexing)
    for node1, node2 in zip(path[:-1], path[1:]):
        G[int(node1) - 1][int(node2) - 1]['weight'] = 1
    
    # Setup the plot
    plt.figure(dpi=dpi, figsize=figsize)
    pos = nx.get_node_attributes(G, 'location')
    
    # Define edge colors and widths based on weight
    edge_colors = ['mediumseagreen' if G[u][v]['weight'] == 1 else 'palegreen' 
                   for u, v in G.edges()]
    edge_widths = [6 if G[u][v]['weight'] == 1 else 1 
                   for u, v in G.edges()]
    
    # Draw the network edges and nodes
    nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color=edge_colors)
    nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color="chartreuse")
    
    plt.title(f'Configuration {conf_number}')
    plt.axis('off')
    
    return G

# Example usage:
conf = 12072
G_result = plot_configuration(conf, G_base, grid_paths)

### Base Adjacency Matrix

In [ ]:
# Keep a copy of the base network
G_base_clean = G_base.copy()

# Set all weights to 1 in the base network
for u, v in G_base.edges():
    G_base[u][v]['weight'] = 1

# Obtain the adjacency matrix
A = nx.adjacency_matrix(G_base).todense()

# Plot the adjacency matrix as a heatmap
plt.figure(dpi=100, figsize=(4, 4))
sns.heatmap(A, cmap='viridis', annot=False, cbar=False)
plt.title('Base Adjacency Matrix')
plt.show()

### Define Utility Functions

In [ ]:
@njit
def compute_energy(Abase, W, p):
    """
    Compute the energy of a configuration.
    
    Parameters:
        Abase : 2D array
            Base adjacency matrix.
        W : 2D array
            Weights matrix for the configuration.
        p : 1D array
            Polarity sequence.
            
    Returns:
        energy : float
            The computed energy.
    """
    e = np.array([-1, 0, 0, -2.3, 0])
    energy = 0.0
    n = len(p)
    
    for i in range(n):
        for j in range(n):
            energy += (1 - W[i, j]) * Abase[i, j] * e[p[i] + p[j]]
    
    return energy

@njit
def compute_energy3(Abase, W, p):
    """
    Compute the energy of a configuration using an alternative energy array.
    
    Parameters:
        Abase : 2D array
            Base adjacency matrix.
        W : 2D array
            Weights matrix for the configuration.
        p : 1D array
            Polarity sequence.
            
    Returns:
        energy : float
            The computed energy.
    """
    e = np.array([-3, -1, -2, 0, -3, -4, -2])
    energy = 0.0
    n = len(p)
    
    for i in range(n):
        for j in range(n):
            energy += (1 - W[i, j]) * Abase[i, j] * e[p[i] + p[j]]
    
    return energy

@njit
def get_weights_matrix(index, grid_paths, shape):
    """
    Compute the weights matrix for a given configuration.
    
    Parameters:
        index : int
            Index of the configuration.
        grid_paths : array-like
            Array of all possible paths.
        shape : tuple
            Desired shape of the weights matrix.
            
    Returns:
        weights : 2D array
            The weights matrix with 1 on the edges belonging to the path.
    """
    # Adjust from 1-based to 0-based indexing
    path = grid_paths[index] - 1
    path = path.astype(np.int32)
    
    weights = np.zeros(shape)
    for i in range(len(path) - 1):
        weights[path[i], path[i + 1]] = 1
        weights[path[i + 1], path[i]] = 1
    return weights

def get_polarity_sequence(n):
    """
    Generate a polarity sequence with two possible values: 1 and -1.
    
    Parameters:
        n : int
            Length of the sequence.
            
    Returns:
        np.array of int: The polarity sequence.
    """
    polarity = [1, -1]
    return np.array([random.choice(polarity) for _ in range(n)])

def get_polarity_sequence3(n):
    """
    Generate a polarity sequence with three possible values: 2, 0, and -1.
    
    Parameters:
        n : int
            Length of the sequence.
            
    Returns:
        np.array of int: The polarity sequence.
    """
    polarity = [2, 0, -1]
    return np.array([random.choice(polarity) for _ in range(n)])

def get_base_adjacency_matrix(G_base):
    """
    Reset all edge weights in G_base to 1 and return the adjacency matrix.
    
    Parameters:
        G_base : networkx.Graph
            Base graph.
            
    Returns:
        A : matrix
            Dense adjacency matrix.
    """
    for u, v in G_base.edges():
        G_base[u][v]['weight'] = 1
    A = nx.adjacency_matrix(G_base).todense()
    return A

### Apply the Algorithm (Full Configuration Set)

In [ ]:
# Parameters and initializations
n_pol_sequences = 2**17  # Adjust as needed (e.g. 2**17)
n_paths = len(grid_paths)
confs_count2 = np.zeros(n_paths)
confs_count3 = np.zeros(n_paths)
# Uncomment to save energy arrays if desired:
# energies2_all = np.zeros((n_pol_sequences, n_paths))
# energies3_all = np.zeros((n_pol_sequences, n_paths))
Abase = get_base_adjacency_matrix(G_base)

# Uncomment to profile the code
# profiler = cProfile.Profile()
# profiler.enable()

for i in tqdm(range(n_pol_sequences)):
    energies2 = np.zeros(n_paths)
    energies3 = np.zeros(n_paths)
    
    # Generate polarity sequences for both energy evaluations
    p2 = get_polarity_sequence(36)
    p3 = get_polarity_sequence3(36)
    
    for j in range(n_paths):
        W = get_weights_matrix(j, grid_paths, (36, 36))
        energies2[j] = compute_energy(Abase, W, p2)
        energies3[j] = compute_energy3(Abase, W, p3)
    
    # Uncomment if saving energy history:
    # energies2_all[i] = energies2
    # energies3_all[i] = energies3
    
    # Find the configuration with minimum energy
    min_energy2 = np.min(energies2)
    min_energy3 = np.min(energies3)
    
    # Update the configuration counts (multiple minima possible)
    confs_count2[np.where(energies2 == min_energy2)[0]] += 1
    confs_count3[np.where(energies3 == min_energy3)[0]] += 1

# Uncomment to save the full energy arrays:
# np.save('energies2_all.npy', energies2_all)
# np.save('energies3_all.npy', energies3_all)
np.save('confs_count2.npy', confs_count2)
np.save('confs_count3.npy', confs_count3)

# Uncomment to log profiling results
# profiler.disable()
# profiler.print_stats(sort='cumtime')

### Load and Visualize Results (Full Set)

In [ ]:
# Uncomment if you have previously saved energies:
# energies2_all = np.load('energies2_all.npy')
# energies3_all = np.load('energies3_all.npy')

# For example, loading a larger count file:
confs_count2 = np.load('confs_count2.npy')
confs_count3 = np.load('confs_count3.npy')

# Identify the 1000 most common configurations
idx2 = np.argsort(confs_count2)[::-1][:1000]
idx3 = np.argsort(confs_count3)[::-1][:1000]
grid_paths2_1000 = grid_paths[idx2]
grid_paths3_1000 = grid_paths[idx3]

# Plot ranking for 2 polarities
most_common_idx2 = np.where(confs_count2 == np.max(confs_count2))[0]
print(f"Most common configurations (2 pol): {most_common_idx2}")

order2 = np.argsort(confs_count2)[::-1]
rank_confs2 = confs_count2[order2]
plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), rank_confs2, s=5, color='indigo')
plt.xlabel('Configuration')
plt.ylabel('Count')
plt.title('Configurations Count (2 polarities)')
plt.show()

# Plot ranking for 3 polarities
most_common_idx3 = np.where(confs_count3 == np.max(confs_count3))[0]
print(f"Most common configurations (3 pol): {most_common_idx3}")

order3 = np.argsort(confs_count3)[::-1]
rank_confs3 = confs_count3[order3]
plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), rank_confs3, s=5, color='mediumseagreen')
plt.xlabel('Configuration')
plt.ylabel('Count')
plt.title('Configurations Count (3 polarities)')
plt.show()

# Plot one of the most common configurations (2 polarities)
G = G_base_clean.copy()
conf = most_common_idx2[0]
path = grid_paths[conf]
for i in range(len(path) - 1):
    node1 = int(path[i]) - 1
    node2 = int(path[i + 1]) - 1
    G[node1][node2]['weight'] = 1
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base_clean, 'location')
nx.draw_networkx_edges(G, pos,
                       width=[6 if G[u][v]['weight'] == 1 else 1 for u, v in G.edges()],
                       edge_color=['mediumseagreen' if G[u][v]['weight'] == 1 else 'palegreen' for u, v in G.edges()])
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="chartreuse")
plt.title('Most Common Configuration (2 polarities)')
plt.axis('off')
plt.show()

# Plot one of the most common configurations (3 polarities)
G = G_base_clean.copy()
conf = most_common_idx3[0]
path = grid_paths[conf]
for i in range(len(path) - 1):
    node1 = int(path[i]) - 1
    node2 = int(path[i + 1]) - 1
    G[node1][node2]['weight'] = 1
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base_clean, 'location')
nx.draw_networkx_edges(G, pos,
                       width=[6 if G[u][v]['weight'] == 1 else 1 for u, v in G.edges()],
                       edge_color=['indigo' if G[u][v]['weight'] == 1 else 'blueviolet' for u, v in G.edges()])
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="plum")
plt.title('Most Common Configuration (3 polarities)')
plt.axis('off')
plt.show()

### Analysis: 1000 Repeated Configurations with Longer Polarization Sequences

In [ ]:
n_pol_sequences = 2**20  # Adjust as needed (e.g. 2**20)
n_paths = len(grid_paths2_1000)
confs_count2 = np.zeros(n_paths)
confs_count3 = np.zeros(n_paths)
Abase = get_base_adjacency_matrix(G_base)

for i in tqdm(range(n_pol_sequences)):
    energies2 = np.zeros(n_paths)
    energies3 = np.zeros(n_paths)
    
    p2 = get_polarity_sequence(36)
    p3 = get_polarity_sequence3(36)
    
    for j in range(n_paths):
        W2 = get_weights_matrix(j, grid_paths2_1000, (36, 36))
        W3 = get_weights_matrix(j, grid_paths3_1000, (36, 36))
        energies2[j] = compute_energy(Abase, W2, p2)
        energies3[j] = compute_energy3(Abase, W3, p3)
    
    min_energy_confs2 = np.argmin(energies2)
    min_energy_confs3 = np.argmin(energies3)
    confs_count2[min_energy_confs2] += 1
    confs_count3[min_energy_confs3] += 1

np.save('confs_count2_1000.npy', confs_count2)
np.save('confs_count3_1000.npy', confs_count3)

# Load results and select the top 100 configurations
confs_count2 = np.load('confs_count2_1000.npy')
confs_count3 = np.load('confs_count3_1000.npy')
n_paths = len(grid_paths2_1000)
idx2 = np.argsort(confs_count2)[::-1][:100]
idx3 = np.argsort(confs_count3)[::-1][:100]
grid_paths2_100 = grid_paths2_1000[idx2]
grid_paths3_100 = grid_paths3_1000[idx3]

most_common_idx2 = np.where(confs_count2 == np.max(confs_count2))[0]
print(f"Most common configurations (2 pol): {most_common_idx2}")
order2 = np.argsort(confs_count2)[::-1]
rank_confs2 = confs_count2[order2]
plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), rank_confs2, s=5, color='indigo')
plt.xlabel('Configuration')
plt.ylabel('Count')
plt.title('Configurations Count (2 polarities)')

most_common_idx3 = np.where(confs_count3 == np.max(confs_count3))[0]
print(f"Most common configurations (3 pol): {most_common_idx3}")
order3 = np.argsort(confs_count3)[::-1]
rank_confs3 = confs_count3[order3]
plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), rank_confs3, s=5, color='mediumseagreen')
plt.xlabel('Configuration')
plt.ylabel('Count')
plt.title('Configurations Count (3 polarities)')

# Plot one of the most common configurations from the reduced set (2 polarities)
G = G_base_clean.copy()
conf = most_common_idx2[0]
path = grid_paths2_1000[conf]
print(path)
for i in range(len(path) - 1):
    node1 = int(path[i]) - 1
    node2 = int(path[i + 1]) - 1
    G[node1][node2]['weight'] = 1
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base_clean, 'location')
nx.draw_networkx_edges(G, pos,
                       width=[6 if G[u][v]['weight'] == 1 else 1 for u, v in G.edges()],
                       edge_color=['indigo' if G[u][v]['weight'] == 1 else 'blueviolet' for u, v in G.edges()])
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="plum")
plt.title('Most Common Configuration (2 polarities)')
plt.axis('off')

# Plot for 3 polarities
G = G_base_clean.copy()
conf = most_common_idx3[0]
path = grid_paths3_1000[conf]
for i in range(len(path) - 1):
    node1 = int(path[i]) - 1
    node2 = int(path[i + 1]) - 1
    G[node1][node2]['weight'] = 1
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base_clean, 'location')
nx.draw_networkx_edges(G, pos,
                       width=[6 if G[u][v]['weight'] == 1 else 1 for u, v in G.edges()],
                       edge_color=['mediumseagreen' if G[u][v]['weight'] == 1 else 'palegreen' for u, v in G.edges()])
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="chartreuse")
plt.title('Most Common Configuration (3 polarities)')
plt.axis('off')

### Analysis: Best 100 Configurations

In [ ]:
n_pol_sequences = 2**24  # Adjust as needed (e.g. 2**24)
n_paths = len(grid_paths2_100)
confs_count2 = np.zeros(n_paths)
confs_count3 = np.zeros(n_paths)
energies2_all = np.zeros((n_pol_sequences, n_paths))
energies3_all = np.zeros((n_pol_sequences, n_paths))
Abase = get_base_adjacency_matrix(G_base)

for i in tqdm(range(n_pol_sequences)):
    energies2 = np.zeros(n_paths)
    energies3 = np.zeros(n_paths)
    
    p2 = get_polarity_sequence(36)
    p3 = get_polarity_sequence3(36)
    
    for j in range(n_paths):
        W2 = get_weights_matrix(j, grid_paths2_100, (36, 36))
        W3 = get_weights_matrix(j, grid_paths3_100, (36, 36))
        energies2[j] = compute_energy(Abase, W2, p2)
        energies3[j] = compute_energy3(Abase, W3, p3)
    
    energies2_all[i] = energies2
    energies3_all[i] = energies3
    min_energy_confs2 = np.argmin(energies2)
    min_energy_confs3 = np.argmin(energies3)
    confs_count2[min_energy_confs2] += 1
    confs_count3[min_energy_confs3] += 1

np.save('energies2_all_100.npy', energies2_all)
np.save('energies3_all_100.npy', energies3_all)
np.save('confs_count2_100.npy', confs_count2)
np.save('confs_count3_100.npy', confs_count3)

# %%
# Load and analyze the results for best 100 configurations
confs_count2 = np.load('confs_count2_100.npy')
confs_count3 = np.load('confs_count3_100.npy')
n_paths = 100

most_common_idx2 = np.where(confs_count2 == np.max(confs_count2))[0]
print(f"Most common configurations (2 pol): {most_common_idx2}")
order2 = np.argsort(confs_count2)[::-1]
rank_confs2 = confs_count2[order2]
plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), rank_confs2, s=5, color='indigo')
plt.xlabel('Configuration')
plt.ylabel('Count')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
plt.title('Configurations Count (2 polarities)')

most_common_idx3 = np.where(confs_count3 == np.max(confs_count3))[0]
print(f"Most common configurations (3 pol): {most_common_idx3}")
order3 = np.argsort(confs_count3)[::-1]
rank_confs3 = confs_count3[order3]
plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), rank_confs3, s=5, color='mediumseagreen')
plt.xlabel('Configuration')
plt.ylabel('Count')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
plt.title('Configurations Count (3 polarities)')

# Plot the most common configuration (2 polarities)
G = G_base_clean.copy()
conf = most_common_idx2[0]
path = grid_paths2_100[conf]
print(path)
for i in range(len(path) - 1):
    node1 = int(path[i]) - 1
    node2 = int(path[i + 1]) - 1
    G[node1][node2]['weight'] = 1
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base_clean, 'location')
nx.draw_networkx_edges(G, pos,
                       width=[6 if G[u][v]['weight'] == 1 else 1 for u, v in G.edges()],
                       edge_color=['indigo' if G[u][v]['weight'] == 1 else 'blueviolet' for u, v in G.edges()])
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="plum")
plt.title('Most Common Configuration (2 polarities)')
plt.axis('off')

# Plot the most common configuration (3 polarities)
G = G_base_clean.copy()
conf = most_common_idx3[0]
path = grid_paths3_100[conf]
for i in range(len(path) - 1):
    node1 = int(path[i]) - 1
    node2 = int(path[i + 1]) - 1
    G[node1][node2]['weight'] = 1
plt.figure(dpi=100, figsize=(5, 5))
pos = nx.get_node_attributes(G_base_clean, 'location')
nx.draw_networkx_edges(G, pos,
                       width=[6 if G[u][v]['weight'] == 1 else 1 for u, v in G.edges()],
                       edge_color=['mediumseagreen' if G[u][v]['weight'] == 1 else 'palegreen' for u, v in G.edges()])
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="chartreuse")
plt.title('Most Common Configuration (3 polarities)')
plt.axis('off')

# Load energy arrays and compute statistics
energies2_all = np.load('energies2_all_100.npy')
energies3_all = np.load('energies3_all_100.npy')

most_common_idx2 = np.where(confs_count2 == np.max(confs_count2))[0]
most_common_idx3 = np.where(confs_count3 == np.max(confs_count3))[0]

mean_energies2 = np.mean(energies2_all, axis=0)
mean_energies3 = np.mean(energies3_all, axis=0)

min_mean_energy2 = np.min(mean_energies2)
min_mean_energy3 = np.min(mean_energies3)
min_mean_energy_confs2 = np.where(mean_energies2 == min_mean_energy2)[0]
min_mean_energy_confs3 = np.where(mean_energies3 == min_mean_energy3)[0]

print(f"Configuration with minimum mean energy (2 polarities): {min_mean_energy_confs2}")
print(f"Configuration with minimum mean energy (3 polarities): {min_mean_energy_confs3}")

plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), mean_energies2, s=3, color='indigo', alpha=0.9)
plt.xlabel('Configuration')
plt.ylabel('Mean Energy')
plt.ticklabel_format(style='plain', axis='y')
plt.title('Mean Energy per Configuration (2 polarities)')

plt.figure(dpi=100, figsize=(6, 4))
plt.scatter(range(n_paths), mean_energies3, s=3, color='mediumseagreen', alpha=0.9)
plt.xlabel('Configuration')
plt.ylabel('Mean Energy')
plt.ticklabel_format(style='plain', axis='y')
plt.title('Mean Energy per Configuration (3 polarities)')